In [2]:
import pandas as pd
from pandas import Series, DataFrame
import datetime 
from datetime import datetime

In [3]:
disney_p = pd.read_csv('/Users/chereowang27/NYCDSA/Python_Intro_Project/data/disney_plus_shows.csv', header=0)
netflix = pd.read_csv('/Users/chereowang27/NYCDSA/Python_Intro_Project/data/netflix_list.csv', header=0)

#convert added at date from str to datetime, make variable for the year, and month-year
disney_p['added_at'] = disney_p["added_at"].apply(lambda s: datetime.strptime(s, "%d-%b-%y"))
disney_p['added_atyear'] = disney_p['added_at'].apply(lambda s: s.strftime('%y'))
disney_p['added_atmonth_year'] = disney_p['added_at'].apply(lambda s: s.strftime('%b-%y'))
disney_p['added_atmonth_year_new'] = pd.to_datetime(disney_p['added_atmonth_year'], format = '%b-%y')

#convert runtime & imdb_rating to numeric
disney_p['runtime'] = disney_p['runtime'].str.split(' ')
disney_p['runtime'] = [pd.to_numeric(val[0:1]) if hasattr(val, '__iter__') else None for val in disney_p["runtime"]]

disney_p['data'] = 'Disney_plus'
netflix['data'] = 'Netflix'
#disney_p['added_atmonth_year'] =  disney_p['added_atyear'].apply(lambda s: datetime.date(s.month, s.year))

hyphen = disney_p['year'].str.split(pat = '-', expand = True ).loc[30,0][4]
df_year = disney_p['year'].str.split(pat = hyphen, expand = True ) \
            .rename(columns = {0:'yearStart', 1: 'yearEnd'})
       

disney_p = pd.concat([disney_p,df_year], axis = 1)

#count the number of movies in each genre
d_movie = disney_p[disney_p['type']=='movie']

In [12]:
# this checks where the titles from disney plus and netflix titles overlap
# 19 titles in both networks
disney_nf = pd.merge(disney_p[['imdb_id','imdb_rating','title']],
          netflix[['imdb_id','rating','title']],
                  on='imdb_id', 
                  how = 'inner', 
         suffixes= ('_disney','_nf')
        )

# this checks if imdb_rating in disney plus data is same as rating in netflix data. Not all, but will take use them as same metric
disney_nf['imdb_rating'] == disney_nf['rating']

#compare imdb scores of movies in disney & netflix 
d_movie = d_movie.rename(columns = {'imdb_rating': 'rating'})
nf_movie = netflix[netflix['type']=='movie'].rename(columns={'genres':'genre','startYear': 'added_atyear'})
allMovie = pd.concat([d_movie[['imdb_id','rating', 'genre', 'added_atyear','data']],nf_movie[['imdb_id','rating','genre','added_atyear','data']]])
allMovie.reindex()


,imdb_id,rating,genre,added_atyear,data
0,tt0147800,7.3,"Comedy, Drama, Romance",19,Disney_plus
2,tt0115433,5.7,"Adventure, Comedy, Crime, Family",19,Disney_plus
3,tt0324941,5.8,"Animation, Adventure, Comedy, Family, Musical",19,Disney_plus
4,tt0211181,4.9,"Adventure, Comedy, Family",19,Disney_plus
5,tt1846442,6.3,"Comedy, Fantasy, Romance",19,Disney_plus
...,...,...,...,...,...
6209,tt14180214,NaN,Comedy,2021.0,Netflix
6212,tt14179106,NaN,Comedy,NaN,Netflix
6216,tt14781176,NaN,"Horror,Thriller",2021.0,Netflix
6621,tt14813966,NaN,Comedy,2022.0,Netflix


In [34]:
[1] + [2] + [3]

[1, 2, 3]

In [ ]:
sum([1,2,3],)
# start + 1 + 2 + 3

In [36]:
sum(
    [[1],[2],[3]], 
    start = []
)


[1, 2, 3]

In [38]:
tmp_df = allMovie.loc[~allMovie["genre"].isnull()]

pd.DataFrame(sum([
    [(pair[0], gn, pair[2]) for gn in pair[1]]
    for pair in zip(tmp_df["imdb_id"], tmp_df["genre"].str.split(", "), tmp_df["rating"])
], start = []))

,0,1,2
0,tt0147800,Comedy,7.3
1,tt0147800,Drama,7.3
2,tt0147800,Romance,7.3
3,tt0115433,Adventure,5.7
4,tt0115433,Comedy,5.7
...,...,...,...
5460,tt14180214,Comedy,NaN
5461,tt14179106,Comedy,NaN
5462,tt14781176,"Horror,Thriller",NaN
5463,tt14813966,Comedy,NaN


In [ ]:
genres_all = set(allMovie['genre'].str.cat(sep = ', ').replace(' ', '').split(','))

In [14]:
nf_20 = allMovie[(allMovie['data']=='Netflix') & (allMovie['added_atyear'] <2021.0)]

TypeError: '<' not supported between instances of 'str' and 'float'

In [5]:
# This counts the number of titles in each genre
def g_count(col,gn):
    return col.str.contains(gn).sum()

# This generates a data frame of counts of each genre by time (either month or year)
def agg_df(group):
    agg_df = []
    for gn in genres:
        tmp = allMovie[allMovie['data']=='Netflix'].groupby(group).agg(
            **{gn : ('genre', lambda col: g_count(col,gn))}
        )
        agg_df.append(tmp)
    df_agg = pd.concat(agg_df, axis=1).apply(lambda col: col.cumsum())
    return df_agg

In [ ]:
 tmp = allMovie[allMovie['data']=='Netflix'].groupby().median()